# Zeta Entanglement ODE Solver

In [15]:
import numpy as np
from scipy.integrate import solve_ivp
from ode_helpers import state_plotter

ImportError: cannot import name solve_ivp

## ODEs 
\begin{equation}
\partial_u^2 f_q (u) + \Omega_f(u, q) f_q(u) = \frac{S_{Eq}(u) f_q(u)}{q^2 (\epsilon-1)^2}
\end{equation}

\begin{equation}
\partial_u^2 g_q (u) + \Omega_g(u, q) g_q(u) = \frac{S_{Eq}(u) g_q(u)}{q^2 (\epsilon-1)^2}
\end{equation}

\begin{equation}
\partial_u \Lambda_q(u) = i \Omega_{\Lambda}(u, q) f_q(u) g_q(u) \sigma(u)
\end{equation}

where: 
\begin{equation}
S_{Eq}(u) = \frac{\Lambda_q^2(u)}{ (f_q(u) g_q(u))^2} + \frac{\epsilon}{2} c_s^2 q^2 (\epsilon-1)^2 (\partial_u \sigma(u))^2
\end{equation}

\begin{equation}
\Lambda_q(u) = \lambda_q(u) + i c_s \sqrt{\frac{\epsilon}{2}} q (\epsilon -1) f_q(u) g_q(u) \partial_u \sigma(u)
\end{equation}

and 

\begin{equation}
\Omega_f(u, q)  = \frac{c_s^2}{(\epsilon -1)^2} - \frac{(2-\epsilon)}{(\epsilon-1)^2} u^{-2} 
\end{equation}

\begin{equation}
\Omega_g(u, q)  = \frac{c_s^2}{(\epsilon -1)^2} - \frac{(2-\epsilon)}{(\epsilon-1)^2} u^{-2} + \frac{m_{\sigma}^2}{(\epsilon -1)^2 } q^{-\frac{2 \epsilon}{\epsilon-1}} u^{\frac{2}{\epsilon -1}} 
\end{equation}

\begin{equation}
\Omega_{\Lambda}(u, q)  =c_s \frac{\sqrt{2 \epsilon}}{1-\epsilon} u^{\frac{2}{\epsilon -1}} q^{\frac{1+\epsilon}{1-\epsilon}} m_{\sigma}^2 
\end{equation}

The backgound equation for $\sigma(u)$ will be solved analytically in Mathematica. For reference it is:

\begin{equation}
\partial^2_u \sigma(u) + \frac{2}{\epsilon -1} u^{-1} \partial_u \sigma(u) + \frac{m_{\sigma}^2 }{(\epsilon -1)^2} q^{-\frac{2 \epsilon}{\epsilon -1}} u^{\frac{2}{\epsilon-1}} \sigma(u) = 0
\end{equation}

Solution for $\sigma(u)$ with no initial conditions ($C1, C2$ are integration constants):

\begin{equation}
\sigma(u) = 2^{\frac{5}{4} -\frac{ 9}{ 4 \epsilon}} u^{\frac{3-\epsilon}{2(1-\epsilon)}} (1-\epsilon)^{\frac{1}{1-\epsilon}} \left( - \frac{m_{\sigma}^2 q^{-\frac{2 \epsilon}{\epsilon -1}} }{\epsilon^2}  \right)^{\frac{3-\epsilon}{4 \epsilon}} \left( C1\; \text{Bessel}_I \left[\frac{\epsilon-3}{2 \epsilon}, \sqrt{-\frac{m_{\sigma}^2 q^{-\frac{2 \epsilon}{\epsilon-1}} u^{\frac{2 \epsilon}{\epsilon -1}}}{\epsilon^2}}\right] -  i^{\frac{3+\epsilon}{\epsilon}} C2 \;\text{Bessel}_K \left[\frac{\epsilon-3}{2 \epsilon}, \sqrt{-\frac{m_{\sigma}^2 q^{-\frac{2 \epsilon}{\epsilon-1}} u^{\frac{2 \epsilon}{\epsilon -1}}}{\epsilon^2}}\right] \right) 
\end{equation}

## Constaints

- $\epsilon$:  slow roll parameter
- $c_s$: sound speed
- $m_{\sigma}$: mass of $\Sigma$ field

In [7]:
epsilon, c_s, m_sigma = 0.1, 0.99, 1

## Defining functions

## $\sigma(u), \; \sigma'(u)$

In [ ]:
def Sig(u, q):
    return 

In [ ]:
def SigD(u, q):
    return 

## $\Omega_f, \Omega_g, \Omega_{\Lambda} $

In [8]:
def Omega_f(u, q):
    return (c_s**2 - (2- epsilon)/u**2)/((epsilon - 1.)**2)

In [9]:
def Omega_g(u, q):
    return (c_s**2 + (m_sigma**2)* (q**(-2* epsilon /(epsilon -1.))) * (u**(2/(epsilon -1)))  - (2- epsilon)/u**2)/((epsilon - 1.)**2)

In [10]:
def Omega_L(u, q):
    return c_s * np.sqrt(2* epsilon) * (q**((1+epsilon) /(1. -epsilon))) * (u**(2/(epsilon -1))) * (m_sigma**2) / (1. - epsilon)

## $S_{Eq}$

In [17]:
def SEq(u, q, f, g, L):
    return ((L**2)/(f* g)**2) + (epsilon/2.0) *(c_s**2) * (q**2) * ((epsilon -1)**2) * (SigD(u, q))**2

## Differential Equation Function

### Redefining as first order equations 

\begin{equation}
F_q(u) = f'_q(u), \quad \partial_u F_q (u) + \Omega_f(u, q) f_q(u) = \frac{S_{Eq}(u) f_q(u)}{q^2 (\epsilon-1)^2}
\end{equation}

\begin{equation}
G_q(u) = g'_q(u), \quad \partial_u G_q (u) + \Omega_g(u, q) g_q(u) = \frac{S_{Eq}(u) g_q(u)}{q^2 (\epsilon-1)^2}
\end{equation}

### Notation:

- $\Lambda_q : y[0]$
- $f_q : y[1]$
- $F_q : y[2]$
- $g_q : y[3]$
- $G_q : y[4]$


In [18]:
def diff(t, y):
    dydt = [1j * Omega_L(u, q) * y[1] * y[3] * Sig(u, q), y[2], (SEq(u,q,y[1],y[3],y[0]) * y[1]/((q**2)*(epsilon-1)**2)) - Omega_f(u,q) * y[1] , y[4],  (SEq(u,q,y[1],y[3],y[0]) * y[3]/((q**2)*(epsilon-1)**2)) - Omega_g(u,q) * y[3] ]
    return dytd

In [19]:
tspan = np.linspace(-100, -0.0001, 100)

In [ ]:
yinit = [Lam_ini, F_ini, f_ini, G_ini, g_ini]

In [ ]:
sol = solve_ivp(lambda t, y: diff(t, y), [tspan[0], tspan[-1]], yinit, t_eval=tspan)

In [14]:
def f(t, y, c):
    dydt = [c[0]*np.cos(c[1]*t), c[2]*y[0]+c[3]*t]
    return dydt

# %% Define time spans, initial values, and constants
tspan = np.linspace(0, 5, 100)
yinit = [0, -3]
c = [4, 3, -2, 0.5]

# %% Solve differential equation
sol = solve_ivp(lambda t, y: f(t, y, c), [tspan[0], tspan[-1]], yinit, t_eval=tspan)

# %% Plot states
state_plotter(sol.t, sol.y, 1)

NameError: name 'solve_ivp' is not defined